In [1]:
import pandas as pd
import os
import shutil
from google.colab import drive
from sklearn.model_selection import train_test_split
import subprocess
import numpy as np

In [2]:
# Mount Google Drive
mount_dir = os.path.join('/content','gdrive')
drive.mount(mount_dir) 

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
# Set working directory and home directory
os.chdir(os.path.join(mount_dir,'MyDrive','DSCI400','Data Wrangling')) 
home_dir = os.path.join(mount_dir,'MyDrive','DSCI400')

In [4]:
# Import apis
import audio_wrangling_api as wrag_api

In [5]:
# Load coswara demographic datasets
coswara = pd.read_csv(os.path.join(home_dir,'Coswara-Data-master','Coswara-Data-master','combined_data.csv'), index_col = 'id')
coughvid = pd.read_csv(os.path.join(home_dir,'New Audio','metadata_compiled.csv'),index_col = 'uuid')

In [6]:
# Add covid label and split into training, validation, test set
coswara = wrag_api.status_regroup(coswara)
coswara_pos, coswara_neg = wrag_api.pos_neg_split("coswara", coswara)
coswara_X, coswara_id = wrag_api.train_test_dmg(coswara_pos, coswara_neg, 0.2, 0.2, 42)

coughvid_pos, coughvid_neg = wrag_api.pos_neg_split("coughvid", coughvid)
coughvid_X, coughvid_id = wrag_api.train_test_dmg(coughvid_pos, coughvid_neg, 0.2, 0.2, 42)

In [7]:
# Set up paths for audio storage
webm_src = os.path.join(home_dir,'New Audio')
coswara_src = os.path.join(home_dir,'Reorganized Audio')
coughvid_src = os.path.join(home_dir,'New Audio wav/')
target = os.path.join(home_dir,'Regroup Audio New')

In [8]:
# Convert coughvid audios from webm to wav
wrag_api.convert_webms(webm_src, coughvid_src)

Data already exist


In [9]:
# Reorganize audio files based on training, validation, test set and covid labels
wrag_api.train_test_org("coswara", coswara_src, target, coswara_id)
wrag_api.train_test_org("coughvid", coughvid_src, target, coughvid_id)

Data already exist
Data already exist


In [10]:
# Combine shallow and heavy cough in the coswara dataset
coswara_coughs = ['cough-shallow','cough-heavy']
wrag_api.combine_audios(target,coswara_coughs,'cough-coswara')

Data already exist
